<a href="https://colab.research.google.com/github/fabarroso/MVP-ML/blob/main/mvpml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MVP -  Machine Learning
Nome: Fabio de Andrade Barroso

Matricula:4052025000158

Dataset original: https://basedosdados.org/dataset/dbd717cb-7da8-4efd-9162-951a71694541?table=a2e9f998-e2c2-49b7-858a-ae1daef46dc0

**Segurança no Estado de São Paulo - Dados estatísticos da Secretaria de Segurança Pública do Estado de São Paulo.**

O dataset contém informações sobre ocorrências policiais no estado de São Paulo, com diversas variáveis relacionadas a diferentes tipos de crimes (homicídios, furtos, roubos, estupros, etc.) por município, mês e ano.

**Organização:**
Governo de São Paulo

**Cobertura temporal:**
2002 - 2021

**1 - Objetivo**

Analisar o comportamento e a ocorrência de crimes ao longo do tempo, utilizando técnicas de Análise Exploratória de Dados (EDA), pré-processamento e aprendizado de máquina com modelos de regressão para identificar padrões, prever a quantidade de ocorrências criminais e compreender possíveis fatores que influenciam a variação desses indicadores.

**1.1 - Escopo**

Exploração dos dados: Investigar a distribuição, tendências temporais e correlações entre variáveis relacionadas a crimes (como homicídios, roubos, furtos, etc.).

Pré-processamento: Tratar valores ausentes, outliers e realizar transformações (normalização, codificação de variáveis categóricas e imputação) para preparar os dados para os modelos.

Modelagem preditiva (Regressão): Avaliar diferentes algoritmos de regressão (Linear, Random Forest, Gradient Boosting, XGBoost, LightGBM) com validação cruzada e busca de hiperparâmetros, para estimar o número de ocorrências criminais.

Avaliação: Comparar modelos com métricas de regressão (RMSE, MAE e R²) e interpretar variáveis de maior relevância (feature importance) no fenômeno estudado.

**1.2 - Contexto do Problema**

A segurança pública é um tema central em discussões políticas e sociais. A análise de dados de criminalidade pode apoiar o planejamento de políticas públicas e estratégias de prevenção.
Com o uso de modelos de regressão em machine learning, é possível estimar quantitativamente a ocorrência de crimes ao longo do tempo, identificar padrões sazonais, detectar tendências de crescimento ou queda e apontar variáveis que mais influenciam esses resultados. Assim, autoridades e gestores podem direcionar recursos de forma mais eficiente e embasar decisões estratégicas.

**2 - Ambiente**

As bibliotecas foram escolhidas para garantir um fluxo eficiente de análise de dados e modelagem. *Pandas* e *NumPy* são usadas para manipulação e cálculo dos dados. *Matplotlib* e *Seaborn* são utilizadas para visualização e exploração gráfica. *Scikit-learn*  facilita o pré-processamento, criação de modelos, validação cruzada e avaliação de desempenho. Essas ferramentas asseguram uma análise robusta e reprodutível do dataset.

Ao carregar o dataset e realizar experimentos com o modelo, a definição da *seed*   assegura que a divisão entre treino e teste e outras etapas aleatórias, como a inicialização do modelo, sejam consistentes. Isso é crucial para garantir que as avaliações de desempenho do modelo sejam justas e reproduzíveis em diferentes execuções.

Foi utilizado pd.set_option do Pandas, evitando assim que colunas ou textos ficassem truncados.

In [ ]:
## Carga de bibliotecas utilizadas

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import random
import sys
#from scipy.stats import chi2_contingency
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split,cross_val_score,KFold,GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


# Definir o SEED para reprodutibilidade (controle de aleatoriedade)
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

# Ajustando as configurações para exibir todas as colunas e linhas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

# Exibição de resultados
print("Python:", sys.version.split()[0])
print("Seed global:", SEED)

Python: 3.12.11
Seed global: 42


**3 - Dados: carga, entendimento e qualidade**

O dataset foi carregado diretamente de uma URL no GitHub, utilizando pd.read_csv, com a configuração de delimitador para vírgula (,) e codificação adequada para caracteres especiais (ISO-8859-1).

In [ ]:
##Carregamento dos Dados
# URL GitHub
url = 'https://raw.githubusercontent.com/fabarroso/dados_sp_gov_ssp/main/sp_gov_ssp.csv'

# Carregamento do dataset
df = pd.read_csv(url, delimiter=',', encoding='ISO-8859-1')

A exclusão dos últimos quatro meses do último ano pode ser justificada pelo fato de que esses meses estavam completamente vazios (ou seja, não continham registros válidos). Ao remover essas linhas vazias, observamos melhora na qualidade dos dados utilizados para análise, evitando que valores ausentes ou irrelevantes distorçam os resultados.

In [ ]:
# Exibe as primeiras 10 linhas
print(df.head(10).to_string())

print("\n" + "-"*660 + "\n")

# Identificar o último ano no dataset
ultimo_ano = df['ano'].max()

# Identificar os últimos 4 meses desse último ano
meses_para_excluir = [9, 10, 11, 12]  # setembro, outubro, novembro, dezembro

# Criar um DataFrame com os dados que serão excluídos (últimos 4 meses do último ano)
df_eliminado = df[(df['ano'] == ultimo_ano) & (df['mes'].isin(meses_para_excluir))]

# Exibir as linhas eliminadas
print("Linhas Eliminadas:")
print(df_eliminado)

print("\n" + "-"*660 + "\n")

# Exibir a quantidade de linhas eliminadas
print("\nQuantidade de Linhas Eliminadas:", df_eliminado.shape[0])

# Filtrando o DataFrame para excluir os últimos 4 meses do último ano
df_filtrado = df[~((df['ano'] == ultimo_ano) & (df['mes'].isin(meses_para_excluir)))]

    ano  mes  id_municipio           regiao_ssp  homicidio_doloso  numero_de_vitimas_em_homicidio_doloso  homicidio_doloso_por_acidente_de_transito  numero_de_vitimas_em_homicidio_doloso_por_acidente_de_transito  homicidio_culposo_por_acidente_de_transito  homicidio_culposo_outros  tentativa_de_homicidio  lesao_corporal_seguida_de_morte  lesao_corporal_dolosa  lesao_corporal_culposa_por_acidente_de_transito  lesao_corporal_culposa_outras  latrocinio  numero_de_vitimas_em_latrocinio  total_de_estupro  estupro  estupro_de_vulneravel  total_de_roubo_outros  roubo_outros  roubo_de_veiculo  roubo_a_banco  roubo_de_carga  furto_outros  furto_de_veiculo
0  2002    1       3500105  Presidente Prudente               0.0                                    0.0                                        0.0                                                             0.0                                         0.0                       0.0                     0.0                              0.0       

**3.1 Análise exploratória (EDA)**

**4 - Pré Processamento**

**Conclusão**

A presente análise, desenvolvida a partir dos dados oficiais da Secretaria de Segurança Pública do Estado de São Paulo (SSP-SP), buscou compreender padrões criminais e aplicar técnicas de regressão em aprendizado de máquina para prever a ocorrência de homicídios dolosos. O estudo contemplou desde a exploração inicial do conjunto de dados até a avaliação comparativa de diferentes modelos preditivos.

Os resultados obtidos permitem destacar alguns pontos relevantes:

Padrões temporais e sazonais – A ocorrência de crimes apresentou flutuações significativas ao longo do tempo, evidenciando sazonalidade em determinados períodos do ano. A remoção dos últimos meses do conjunto mais recente mostrou-se necessária para evitar distorções causadas por registros incompletos, garantindo maior fidedignidade ao processo de modelagem.

Qualidade e preparação dos dados – O pré-processamento realizado (tratamento de valores ausentes, normalização, winsorização de outliers e codificação de variáveis categóricas) foi determinante para melhorar a consistência estatística dos dados e a robustez dos modelos de regressão aplicados.

Relações entre variáveis – A análise de correlação evidenciou vínculos entre diferentes categorias de crimes e os homicídios dolosos, sugerindo que fenômenos criminais não ocorrem de forma isolada, mas sim interconectada, refletindo dinâmicas sociais e territoriais complexas.

Desempenho dos modelos de regressão – A comparação entre algoritmos mostrou que os métodos baseados em árvores de decisão e boosting (Gradient Boosting, XGBoost e LightGBM) apresentaram resultados superiores em termos de capacidade explicativa (R²) e menores erros preditivos (RMSE e MAE), em relação a modelos lineares tradicionais.

Explicabilidade e aplicação prática – A análise da importância das variáveis reforçou a relevância de fatores temporais e contextuais na previsão de homicídios dolosos. Esses resultados podem servir como suporte a gestores públicos e pesquisadores na formulação de políticas de segurança mais direcionadas, pautadas em evidências empíricas.

De forma geral, conclui-se que a aplicação de técnicas de regressão em aprendizado de máquina é eficaz para o estudo de dados criminais, possibilitando tanto a previsão quantitativa de ocorrências quanto a identificação de fatores de maior impacto. Tais achados reforçam o potencial do uso de ciência de dados na segurança pública, não apenas para compreender o passado, mas também para antecipar tendências e subsidiar decisões estratégicas.

Por fim, cabe destacar que a análise está sujeita a limitações inerentes ao próprio dataset, como a possível subnotificação de ocorrências e a ausência de variáveis socioeconômicas ou demográficas, que poderiam ampliar a capacidade explicativa dos modelos. Recomenda-se, portanto, que estudos futuros explorem bases de dados complementares e enfoquem abordagens multivariadas, de modo a aprofundar a compreensão sobre os determinantes da criminalidade.

In [ ]:
# ==============================================================
# Projeto: Regressão Explicativa - SSP SP (Versão Acadêmica Final)
# ==============================================================

# 1. Ambiente e Reprodutibilidade
!pip install xgboost lightgbm --quiet

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score, learning_curve
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

SEED = 42
np.random.seed(SEED)

# ==============================================================
# 2. Objetivo e Definição do Problema
"""
Objetivo: Analisar a evolução da criminalidade no Estado de São Paulo e
prever a quantidade de homicídios dolosos utilizando regressão e técnicas
de aprendizado de máquina.

Justificativa: A predição pode apoiar políticas públicas de segurança,
otimização de recursos policiais e planejamento estratégico.
"""

# ==============================================================
# 3. Carregar dataset
print("Carregando dataset...")
url = 'https://raw.githubusercontent.com/fabarroso/dados_sp_gov_ssp/main/sp_gov_ssp.csv'
df = pd.read_csv(url)
print("Shape original:", df.shape)
display(df.head())

# ==============================================================
# 3.1 Entendimento e Qualidade dos Dados
print("\nResumo das colunas e tipos de dados:")
display(df.info())
print("\nValores ausentes por coluna (%):")
display((df.isna().sum()/len(df)*100).sort_values(ascending=False))

# ==============================================================
# 4. Remover últimos 4 meses do último ano
ultimo_ano = df['ano'].max()
meses_para_excluir = [9,10,11,12]
linhas_remover = df[(df['ano']==ultimo_ano)&(df['mes'].isin(meses_para_excluir))]
print(f"\nNúmero de linhas a serem removidas: {linhas_remover.shape[0]}")
display(linhas_remover)

df = df.drop(linhas_remover.index).reset_index(drop=True)
print("Shape após remoção últimos 4 meses:", df.shape)

# ==============================================================
# 5. Função para Winsorização de outliers
def winsorize_outliers(df, num_cols):
    df_w = df.copy()
    for col in num_cols:
        Q1 = df_w[col].quantile(0.25)
        Q3 = df_w[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5*IQR
        upper = Q3 + 1.5*IQR
        df_w[col] = np.where(df_w[col] < lower, lower, df_w[col])
        df_w[col] = np.where(df_w[col] > upper, upper, df_w[col])
    return df_w

# ==============================================================
# 6. EDA completo
def eda_regression(df, target):
    print("\n=== Estatísticas Descritivas ===")
    display(df.describe().T)

    # Histograma do target
    plt.figure(figsize=(6,3))
    plt.hist(df[target], bins=20, color='skyblue', edgecolor='black')
    plt.title(f'Histograma - {target}')
    plt.xlabel(target)
    plt.ylabel('Frequência')
    plt.show()

    # Seleção de variáveis numéricas
    num_cols = df.select_dtypes(include=np.number).columns.tolist()
    num_cols.remove(target)

    # Top 5 variáveis para gráficos
    top_num_cols = num_cols[:5]
    for col in top_num_cols:
        plt.figure(figsize=(6,3))
        plt.hist(df[col].dropna(), bins=20, color='lightgreen', edgecolor='black')
        plt.title(f'Histograma - {col}')
        plt.show()

        plt.figure(figsize=(6,3))
        sns.violinplot(x=df[col], color='lightblue')
        plt.title(f'Violin Plot - {col}')
        plt.show()

        # Boxplot
        plt.figure(figsize=(6,3))
        sns.boxplot(x=df[col])
        plt.title(f'Boxplot - {col}')
        plt.show()

    # Heatmap de correlação (Top 10 numéricas mais correlacionadas com o target)
    corr_matrix = df[num_cols + [target]].corr()
    top_corr_cols = corr_matrix[target].abs().sort_values(ascending=False).iloc[1:11].index.tolist()
    plt.figure(figsize=(10,6))
    sns.heatmap(df[top_corr_cols + [target]].corr(), annot=True, fmt='.2f', cmap='coolwarm', vmin=-1, vmax=1)
    plt.title('Matriz de Correlação - Top 10 numéricas')
    plt.show()

    # Valores ausentes
    missing = (df.isna().sum()/len(df)*100).sort_values(ascending=False)
    print("\n=== Valores Ausentes (%) ===")
    display(missing[missing>0])

    return df, num_cols

df, num_cols = eda_regression(df, 'homicidio_doloso')

# Aplicando Winsorização para outliers
df = winsorize_outliers(df, num_cols)

# ==============================================================
# 7. Definição de features e target
target = 'homicidio_doloso'

# ==============================================================
# 7.1 Remover colunas altamente correlacionadas
corr_matrix = df[num_cols].corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [col for col in upper.columns if any(upper[col] > 0.9)]
print(f"Colunas removidas por alta correlação (>0.9): {to_drop}")

# Definir X removendo target e colunas altamente correlacionadas
X = df.drop(columns=[target] + to_drop)
y = df[target]

# Atualizar colunas numéricas após remoção
num_cols = [col for col in num_cols if col not in to_drop]

# ==============================================================
# 8. Divisão treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)
print("Treino:", X_train.shape, "Teste:", X_test.shape)

# ==============================================================
# 9. Pré-processamento
cat_cols = X_train.select_dtypes(include='object').columns.tolist()

numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy='median')),  # 1. Missing values numéricos
    ("scaler", StandardScaler())                    # 2. Normalização
])

categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy='most_frequent')),  # 1. Missing values categóricos
    ("onehot", OneHotEncoder(handle_unknown='ignore'))     # 3. OneHotEncoding
])

preprocessor = ColumnTransformer([
    ("num", numeric_transformer, num_cols),
    ("cat", categorical_transformer, cat_cols)
])

# ==============================================================
# 10. Modelos candidatos
models_params = {
    "LinearRegression": {"model": LinearRegression(), "params": {}},
    "RandomForest": {"model": RandomForestRegressor(random_state=SEED),
                     "params": {"model__n_estimators": [50,100], "model__max_depth": [5,10]}},
    "GradientBoosting": {"model": GradientBoostingRegressor(random_state=SEED),
                         "params": {"model__n_estimators":[50,100], "model__learning_rate":[0.05,0.1], "model__max_depth":[3,5]}},
    "XGBoost": {"model": XGBRegressor(random_state=SEED, verbosity=0),
                "params": {"model__n_estimators":[50,100], "model__learning_rate":[0.05,0.1], "model__max_depth":[3,5]}},
    "LightGBM": {"model": LGBMRegressor(random_state=SEED, verbose=-1),
                 "params": {"model__n_estimators":[50,100], "model__learning_rate":[0.05,0.1], "model__max_depth":[5]}}
}

# ==============================================================
# 11. Treino, RandomizedSearchCV e Avaliação
best_models = {}
metrics = []

for name, mp in models_params.items():
    print(f"\n=== Treino e Otimização {name} ===")
    pipe = Pipeline([("prep", preprocessor), ("model", mp["model"])])

    if mp["params"]:
        search = RandomizedSearchCV(pipe, mp["params"], n_iter=4, cv=3, scoring='r2', random_state=SEED, n_jobs=-1)
        search.fit(X_train, y_train)
        best_models[name] = search.best_estimator_
        print(f"Melhor R² CV: {search.best_score_:.3f}")
        print(f"Melhores parâmetros: {search.best_params_}")
    else:
        pipe.fit(X_train, y_train)
        best_models[name] = pipe
        score = cross_val_score(pipe, X_train, y_train, cv=3, scoring='r2').mean()
        print(f"R² CV: {score:.3f}")

    # Previsão e métricas
    y_pred = best_models[name].predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    metrics.append({"Model": name, "RMSE": rmse, "MAE": mae, "R2": r2})

# ==============================================================
# 12. Tabela resumida de hiperparâmetros otimizados (após RandomizedSearchCV)
hyperparams_summary = []

for name, mp in models_params.items():
    if mp["params"]:  # Modelos que tiveram RandomizedSearchCV
        best_params = best_models[name].named_steps["model"].get_params()
        filtered_params = {k: v for k, v in best_params.items() if k in [p.replace("model__", "") for p in mp["params"].keys()]}
        hyperparams_summary.append({"Model": name, **filtered_params})
    else:
        hyperparams_summary.append({"Model": name, "Info": "Sem hiperparâmetros ajustáveis"})

hyperparams_df = pd.DataFrame(hyperparams_summary)
print("\n=== Hiperparâmetros Otimizados ===")
display(hyperparams_df)

# ==============================================================
# 13. Comparativo final
metrics_df = pd.DataFrame(metrics).sort_values("R2", ascending=False)
display(metrics_df)

metrics_melt = metrics_df.melt(id_vars='Model', value_vars=['RMSE','MAE','R2'], var_name='Metric', value_name='Value')
plt.figure(figsize=(10,6))
sns.barplot(x='Model', y='Value', hue='Metric', data=metrics_melt)
plt.title("Comparativo de Performance dos Modelos")
plt.xticks(rotation=45)
plt.show()

# ==============================================================
# 14. Melhor modelo, análise de erros e Learning Curves
best_name = metrics_df.iloc[0]['Model']
best_model = best_models[best_name]
print(f"Melhor modelo selecionado: {best_name}")

# Real vs Predito
y_pred = best_model.predict(X_test)
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred, alpha=0.3)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel("Real")
plt.ylabel("Predito")
plt.title(f"Real vs Predito - {best_name}")
plt.show()

# Feature Importance (se aplicável)
try:
    model_obj = best_model.named_steps["model"]
    if hasattr(model_obj, "feature_importances_"):
        cat_features = best_model.named_steps["prep"].named_transformers_["cat"].named_steps["onehot"].get_feature_names_out(cat_cols)
        feature_names = num_cols + list(cat_features)
        importances = model_obj.feature_importances_
        feat_imp = pd.DataFrame({"feature": feature_names, "importance": importances}).sort_values("importance", ascending=False).head(20)
        plt.figure(figsize=(10,6))
        sns.barplot(x="importance", y="feature", data=feat_imp)
        plt.title(f"Top 20 Features - {best_name}")
        plt.show()
except:
    print("Não foi possível calcular feature importance para este modelo.")

# Learning Curves
train_sizes, train_scores, test_scores = learning_curve(best_model, X_train, y_train, cv=3, scoring='r2', n_jobs=-1, train_sizes=np.linspace(0.1,1.0,5))
plt.figure(figsize=(7,5))
plt.plot(train_sizes, train_scores.mean(axis=1), "o-", label="Train R²")
plt.plot(train_sizes, test_scores.mean(axis=1), "o-", label="CV R²")
plt.xlabel("Amostra de Treino")
plt.ylabel("R²")
plt.title(f"Learning Curve - {best_name}")
plt.legend()
plt.show()